In [2]:
!pip install --upgrade --index-url https://pypi.ngc.nvidia.com nvidia-tensorrt

Looking in indexes: https://pypi.ngc.nvidia.com, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 774.4 MB 16 kB/s 
     |████████████████████████████████| 333.1 MB 40 kB/s 
     |████████████████████████████████| 849 kB 53.3 MB/s 
     |████████████████████████████████| 719.3 MB 18 kB/s 
  Created wheel for nvidia-cublas-cu11: filename=nvidia_cublas_cu11-2022.4.8-py3-none-any.whl size=15624 sha256=bc26a9a62859a05a6124afc78be76b20de729fb78df2e9a4b208d1d58b43247f
  Stored in directory: /root/.cache/pip/wheels/e2/c3/94/1ffd5bac267cfdc2b222a4ec6915278ef18a028a916b9a5ac3
  Created wheel for nvidia-cuda-runtime-cu11: filename=nvidia_cuda_runtime_cu11-2022.4.25-py3-none-any.whl size=15696 sha256=e581942e5586b7b0afae3670560936c450b0561e7f62c05e29adddc2fd168472
  Stored in directory: /root/.cache/pip/wheels/df/fe/2b/e553db7867508b2268b14ac194e9ac5b3f51f21316c282c96c
  Created wheel for nvidia-cudnn-cu11: filename=nvidia_cudnn_cu11-2022.5.19-py3-none

In [3]:
import tensorrt as trt

trt.__version__

'8.4.1.5'

In [43]:
!pip install python-mnist
import tensorflow as tf
import sys
fashion_mnist_path = "./fashion-mnist/"
sys.path.insert(0, fashion_mnist_path + 'utils')
from tensorflow.keras.datasets import fashion_mnist
import mnist.loader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install tf2onnx onnxsim pycuda onnxruntime-gpu
import numpy as np
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.datasets import mnist
import time
import tf2onnx

import onnx
from onnxsim import simplify

import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda

import os.path


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111.0 MB 47 kB/s 


In [18]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2500)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [44]:
#def load_fashion_mnist_db(classNames):
def load_fashion_mnist_db():
    # Get time
    preProcessStartTime = time.time()

    #https://github.com/zalandoresearch/fashion-mnist

    # Load Fashion MNIST data
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    #X_train, y_train = mnist_reader.load_mnist(fashion_mnist_path + '/data/fashion', kind='train')
    #X_test, y_test = mnist_reader.load_mnist(fashion_mnist_path + '/data/fashion', kind='t10k')

    # (trainX, trainY), (testX, testY) = mnist.load_data()

    # Sanity test for the data loading
    print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
    print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))

    # Reshape line vector shape into image shape
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

    # one hot encode target values
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    # Normalize input data into [0, 1]
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    preProcessEndTime = time.time()
    print(f"Data set pre processing time (CPU) is - {(preProcessEndTime - preProcessStartTime) / 1e-3} ms")

    return X_train, y_train, X_test, y_test


In [45]:
X_train, y_train, X_test, y_test = load_fashion_mnist_db()


4431872/4422102 [==============================] - 0s 0us/step
Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)
Data set pre processing time (CPU) is - 1066.4234161376953 ms


In [46]:
print("Done converting to float32 normalized values")

Done converting to float32 normalized values


In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dense(10, activation='softmax')
])


In [48]:
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])


In [49]:
model.fit(X_train, y_train, epochs=2)


Epoch 1/2
1875/1875 [==============================] - 9s 3ms/step - loss: 0.4337 - accuracy: 0.8437
Epoch 2/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2887 - accuracy: 0.8950


In [50]:
s1 = time.time()
resOrg = np.argmax(model.predict(np.asanyarray([X_test[0]]),), axis=-1)
e1 = time.time()


In [51]:
input_signature = [tf.TensorSpec([1, 28, 28, 1], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [52]:
print("Saved onnx model to model.onnx")

Saved onnx model to model.onnx


In [53]:
# load your predefined ONNX model
model = onnx.load('model.onnx')


In [54]:
# convert model
model_simp, check = simplify(model)


In [55]:
# save simplified model
with open("model-simplified.onnx", "wb") as f:
    f.write(model_simp.SerializeToString())


In [56]:
print("Saved onnx model to model-simplified.onnx")


Saved onnx model to model-simplified.onnx


In [57]:
logger = trt.Logger(trt.Logger.VERBOSE)
logger.min_severity = trt.Logger.Severity.VERBOSE
EXPLICIT_BATCH = []


In [58]:
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        """Within this context, host_mom means the cpu memory and device means the GPU memory
        """
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()


In [59]:
def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream


In [70]:
def load_engine(engine_path):
    with open(engine_path, "rb") as f, trt.Runtime(logger) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

In [60]:
print('trt version', trt.__version__)


trt version 8.4.1.5


In [61]:
if not os.path.exists('optimized.trt'):
    if trt.__version__[0] >= '7':
        EXPLICIT_BATCH.append(1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

    with trt.Builder(logger) as builder, builder.create_builder_config() as config, builder.create_network(*EXPLICIT_BATCH) as network, trt.OnnxParser(network,
                                                                                                            logger) as parser:
        #config.max_workspace_size = 1 << 28
        #builder.max_workspace_size = 1 << 28
        builder.max_batch_size = 1
        #builder.fp16_mode = False

        with open('model-simplified.onnx', 'rb') as f:
            if not parser.parse(f.read()):
                for error in range(parser.num_errors):
                    print(parser.get_error(error))
            else:
                print("Parsed ONNX successfully")

        # reshape input from 32 to 1
        shape = list(network.get_input(0).shape)
        print("building tensorrt engine")
        #engine = builder.build_cuda_engine(network)
        engine = builder.build_engine(network, config)
        print("Saving serialized model")
        with open('optimized.trt', 'wb') as f:
            f.write(engine.serialize())

        print("Done")
else:
    engine = load_engine('optimized.trt')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Use build_serialized_network instead.


Parsed ONNX successfully
building tensorrt engine
Saving serialized model
Done


In [62]:
ctx = engine.create_execution_context()


In [63]:
# Allocate buffers for input and output
inputs, outputs, bindings, stream = allocate_buffers(engine)  # input, output: host # bindings


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  import sys


In [64]:
def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer data from CPU to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]


In [65]:
def postprocess_the_outputs(h_outputs, shape_of_output):
    h_outputs = h_outputs.reshape(*shape_of_output)
    return h_outputs


In [66]:
max_batch_size = 1 # The batch size of input mush be smaller the max_batch_size once the engine is built


In [67]:
np.copyto(inputs[0].host, np.asanyarray(X_train[0]).ravel())


In [68]:
shape_of_output = (1, 10)
s2 = time.time()
resOpt = np.argmax(postprocess_the_outputs(do_inference(ctx, bindings, inputs, outputs, stream)[0], shape_of_output))
e2 = time.time()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Use execute_async_v2 instead.
  """


In [69]:
print("Original model time: " + str(e1 - s1), " , result - ", resOrg)
print("Optimized model: " + str(e2 - s2), " , result - ", resOpt)

print('Done!')

Original model time: 0.12091970443725586  , result -  [9]
Optimized model: 0.007019996643066406  , result -  9
Done!
